# Extract English source sentences for NER

___

In [37]:
import pandas as pd

In [38]:
en_de = pd.read_csv("input/train.ende.df.short.tsv", sep='\t', index_col="index")

In [39]:
en_de.head()

,original,translation,scores,mean,z_scores,z_mean,model_scores
index,,,,,,,
0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,"[100, 100, 100]",100.0,"[0.9553316582231767, 1.5523622744336512, 0.850...",1.119409,-0.102446
1,"However, a disappointing ninth in China meant ...",Eine enttäuschende Neunte in China bedeutete j...,"[61, 70, 87]",72.66666666666667,"[-1.1951539386222017, 0.04217636328434989, -0....",-0.488591,-0.431046
2,"In his diary, Chase wrote that the release of ...","In seinem Tagebuch, Chase schrieb, dass die Ve...","[52, 45, 49]",48.666666666666664,"[-1.6914198455865197, -1.2163118960067345, -3....",-2.207007,-0.351719
3,Heavy arquebuses mounted on wagons were called...,Schwere Arquebuses auf Waggons montiert wurden...,"[61, 55, 85]",67.0,"[-1.1951539386222017, -0.7129165922903007, -0....",-0.799946,-0.432696
4,Once North Pacific salmon die off after spawni...,Sobald der nordpazifische Lachs nach dem Laich...,"[91, 89, 87]",89.0,"[0.45906575125885857, 0.998627440345574, -0.31...",0.381633,-0.392975


In [40]:
en_de = en_de.head(100)

**Extract only first 100 lines. Put these in `train_en.txt`**

In [41]:
with open("tmp/train_en.txt", "w", encoding='utf-8') as f:
    for index, row in en_de.iterrows():
        f.write(row.original + '\n')

# Swapping NE's with NEXXX


In [42]:
#Import SpaCy
import spacy
from spacy import displacy
from collections import Counter
nlp = spacy.load("en_core_web_sm")

In [43]:
#Instantiate a dictionary
d={}

#List of NE's we want:
named_entity_labels = ["PERSON", "ORG", "PRODUCT", "LOC", "WORK_OF_ART"]

FILE = "tmp/train_en.txt"
#Read our lines and put NE's in dictionary:
with open(FILE, "r", encoding='utf-8') as f:
    i = 0
    j = 1
    for line in f:
        doc=nlp(line)
        for X in doc.ents:
            if X.label_ in named_entity_labels:
                l = len(str(X))
                k = str(j) + ((l -(2 + len(str(j)))) * "*")
                d[X.text] = ("NE{}".format(k),"{}".format(i),("{}".format(X.start_char), "{}".format(X.end_char)))
                j+=1
        i+=1
    f.close()

# We have a dictionary of NE's
**This dictionary contains:**<br>
1) The English Source<br>
2) The NEXXX value it will receive<br>
3) The line number it is in<br>
4) The start and end letter of that name in terms of the sentence.

In [44]:
#Open a new file.
#read from old file.
#For each sentence, match the sentence index with the NE's sentence number in the dictionary
#If they match, let the start and end position of that NE within the sentence be equal to NEXXX
#Write this altered sentence to the new file

#FOR EXAMPLE:
#             source = "José Ortega y Gasset visited Husserl at Freiburg in 1934."
#             target = "NE1***************** visited NE2**** at Freiburg in 1934"

FILE="tmp/train_en.txt"
FILE2="output/train_en_swapped_ne.txt"
with open(FILE2, "w", encoding='utf-8') as f2:
    with open(FILE, "r", encoding='utf-8')as f:
        line_tracker = 0
        lines = f.readlines()
        for line in lines:
            line = line.strip('\n')
            for entry in d:
                if(int(d[entry][1]) == line_tracker):
                    start, stop = int(d[entry][2][0]), int(d[entry][2][1])
                    before, after = line[0:start], line[stop+1:]
                    line = " ".join([before, d[entry][0], after])
            f2.write(line + "\n")
            line_tracker+=1

In [45]:
#Here is the first few lines of output for you to see;

with open(FILE2, "r", encoding='utf-8') as f:
    i = 0
    lines = f.readlines()
    for i in range(0,10):
        print(lines[i])

 NE1***************** visited NE2****  at Freiburg in 1934.

However, a disappointing ninth in China meant that he dropped back to sixth in the standings.

In his diary,  NE3** wrote that the release of NE4**  an NE5**** l "…was like gall and wormwood to me.

Heavy arquebuses mounted on wagons were called arquebus à croc.

Once  NE6********** salmon die off after spawning, usually local bald eagles eat salmon carcasses almost exclusively.

They engaged in crossfire at Guamaní River Bridge,  NE7** and Silva Heights and finally at NE8******************* .

Several WHA teams courted  NE9****  notably NE10*******************  and the Birmingham Bulls.

16  NE11****** in which redshirt freshman NE12*****  threw for 280 yards and three touchdowns in his first career start as Miami's quarterback.

On 12 April 1983,  NE13********************************* designated the 69th Regiment Armory an official New York City landmark.

After that, Chinese junks came to northern  NE14** with cargoes of s